# **Fine-tuning XLSR-Wav2Vec2 for Multi-Lingual ASR with 🤗 Transformers**

In [ ]:
!nvidia-smi

Mon Oct 24 08:01:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    46W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         167G   41G  127G  25% /
tmpfs            64M     0   64M   0% /dev
shm              41G     0   41G   0% /dev/shm
/dev/root       2.0G  1.1G  910M  54% /sbin/docker-init
tmpfs            42G   48K   42G   1% /var/colab
/dev/sda1       174G   42G  132G  25% /opt/bin/.nvidia
tmpfs            42G     0   42G   0% /proc/acpi
tmpfs            42G     0   42G   0% /proc/scsi
tmpfs            42G     0   42G   0% /sys/firmware
drive           167G   47G  121G  28% /content/gdrive


In [ ]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
%%capture
!pip install datasets
!pip install transformer == 4.11.3
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install numpy

## Prepare Data, Tokenizer, Feature Extractor

### Create Wav2Vec2CTCTokenizer

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#mais est-ce que c'est vraiment une bonne choses d'ignorer les warning? Des fois les warnings signifient des potenciels bugs 

In [ ]:
import datasets
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "vi", split="train")
common_voice_test = load_dataset("common_voice", "vi", split="test")

Generating train split:   0%|          | 0/221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/198 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/870 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/619 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/78 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/vi/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e. Subsequent calls will reuse this data.


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 221
})

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=15):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]), num_examples=15)

,sentence
0,"Như không khí, như màu xanh lá cỏ"
1,và nhất quyết không thưa
2,Bà bói liền đáp
3,Thơ tình cuối mùa thu
4,Nên hà ni đã phải kêu đau đớn
5,lúc đi qua ngã ba
6,Mặc cho nhân viên gọi với theo
7,Cũng hết thời gian cho kuman ăn
8,giờ thành con ma dại đến nhà cũng k thể về
9,Đợi đến giờ bà bói làm lễ



 Enlever les ponctuations et aussi mettre tous en minuscules 


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\*\”]' #adapté selon le dataset vietnamien

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/221 [00:00<?, ?ex/s]

  0%|          | 0/198 [00:00<?, ?ex/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]),num_examples=15)

,sentence
0,anh chị chào mào
1,viêt nam cố lên
2,chinh lúc này mới giật mình nhìn sang bên cạnh
3,thanh ơi
4,mẹ không có an tâm
5,mày quay mặt lại đây nói chuyện tử tế với tao xem nào
6,mẹ của thảo nói
7,bởi suy cho cùng tất cả mọi thứ do nhi lựa chọn
8,vẫn nhìn được rõ chiếc xe tải tử thần màu xanh ấy
9,tôi chờ người đến với yêu thương


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"]) #en "one big string"
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True,remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True,remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0])) #merge!

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict #espace à ne pas enlevé et ensuite à changer en |

{'g': 0,
 'ỳ': 1,
 'ă': 2,
 'ì': 3,
 'b': 4,
 'ằ': 5,
 'ỉ': 6,
 'ư': 7,
 'ổ': 8,
 'ũ': 9,
 'õ': 10,
 'l': 11,
 'v': 12,
 'ú': 13,
 'ạ': 14,
 'ẽ': 15,
 'h': 16,
 'ĩ': 17,
 'ẻ': 18,
 'ã': 19,
 'ỡ': 20,
 'ề': 21,
 'ắ': 22,
 'â': 23,
 't': 24,
 'ê': 25,
 'í': 26,
 'ò': 27,
 'ó': 28,
 'ả': 29,
 'à': 30,
 'r': 31,
 'ị': 32,
 'x': 33,
 'ầ': 34,
 'ậ': 35,
 'ể': 36,
 'y': 37,
 'ỗ': 38,
 'n': 39,
 's': 40,
 'ự': 41,
 'ẩ': 42,
 'ờ': 43,
 'é': 44,
 'ơ': 45,
 'c': 46,
 'o': 47,
 'ọ': 48,
 'u': 49,
 'ữ': 50,
 'd': 51,
 'p': 52,
 'á': 53,
 'a': 54,
 'ệ': 55,
 'ố': 56,
 'ừ': 57,
 'i': 58,
 'ử': 59,
 'ễ': 60,
 'ứ': 61,
 'ợ': 62,
 'đ': 63,
 'ý': 64,
 'ế': 65,
 'ẵ': 66,
 'ẫ': 67,
 'ỷ': 68,
 'ở': 69,
 'q': 70,
 'ô': 71,
 'è': 72,
 'ộ': 73,
 'ù': 74,
 ' ': 75,
 'ẳ': 76,
 'ấ': 77,
 'ớ': 78,
 'ụ': 79,
 'ủ': 80,
 'e': 81,
 'ỏ': 82,
 'ẹ': 83,
 'ặ': 84,
 'm': 85,
 'ồ': 86,
 'k': 87}

À vérifier et repasser pour enlever tous les SW


In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK_toc]"] = len(vocab_dict)
vocab_dict["[PAD_toc]"] = len(vocab_dict) 
len(vocab_dict)

90

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [ ]:

from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK_toc]", pad_token="[PAD_toc]", word_delimiter_token="|")

Next, we will create the feature extractor.

### Create XLSR-Wav2Vec2 Feature Extractor

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

Mounted at /content/gdrive/


In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-Viet")

Next, we can prepare the dataset.

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-Viet")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Preprocess Data




Attention aux problèmes peuvent être causé par le dossier cache, pour éviter de bug, enpêcher sa créeation


In [ ]:
common_voice_train[0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c750494166ba1048b09922d28c0c120bc31648cf605fbc459c331c60de606dae/cv-corpus-6.1-2020-12-11/vi/clips/common_voice_vi_22402288.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c750494166ba1048b09922d28c0c120bc31648cf605fbc459c331c60de606dae/cv-corpus-6.1-2020-12-11/vi/clips/common_voice_vi_22402288.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
  'sampling_rate': 48000},
 'sentence': 'nhất là lại nhớ đến người phụ nữ dân tộc kia '}

In [ ]:
common_voice_train["path"][0]  #attention ordre d'habitude ne marche pas il faut inverser 

'/root/.cache/huggingface/datasets/downloads/extracted/c750494166ba1048b09922d28c0c120bc31648cf605fbc459c331c60de606dae/cv-corpus-6.1-2020-12-11/vi/clips/common_voice_vi_22402288.mp3'

In [ ]:
common_voice_train[0]["audio"] # on veut 16000

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c750494166ba1048b09922d28c0c120bc31648cf605fbc459c331c60de606dae/cv-corpus-6.1-2020-12-11/vi/clips/common_voice_vi_22402288.mp3',
 'array': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 'sampling_rate': 48000}

Let's take a look at `"audio"` again.

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"] #cool c'est fait

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c750494166ba1048b09922d28c0c120bc31648cf605fbc459c331c60de606dae/cv-corpus-6.1-2020-12-11/vi/clips/common_voice_vi_22402288.mp3',
 'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         2.2200273e-05, -1.1045805e-05, -3.1818249e-06], dtype=float32),
 'sampling_rate': 16000}

We don't like the warning message, and there is a way to ignore it (librairie) warnings
warnings.filterwarnings('ignore')

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: gắn bó nhau vì một lối đi chung 
Input array shape: (57216,)
Sampling rate: 16000


Great, it seems like the data is now correctly loaded and resampled. 

It can be heard, that the speakers change along with their speaking rate,accent, and background environment, etc. Overall, the recordings sound acceptably clear though, which is to be expected from a crowd-sourced read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

#0:   0%|          | 0/56 [00:00<?, ?ex/s]

#1:   0%|          | 0/55 [00:00<?, ?ex/s]

#2:   0%|          | 0/55 [00:00<?, ?ex/s]

#3:   0%|          | 0/55 [00:00<?, ?ex/s]

#0:   0%|          | 0/50 [00:00<?, ?ex/s]

#1:   0%|          | 0/50 [00:00<?, ?ex/s]

#2:   0%|          | 0/49 [00:00<?, ?ex/s]

#3:   0%|          | 0/49 [00:00<?, ?ex/s]

Let's apply the data preparation function to all examples.

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the 
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
wer_metric = load_metric("wer")

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `XLSR-Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

Because the dataset is quite small (~6h of training data) and because Common Voice is quite noisy, fine-tuning Facebook's [wav2vec2-large-xlsr-53 checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) seems to require some hyper-parameter tuning. Therefore, I had to play around a bit with different values for dropout, [SpecAugment](https://arxiv.org/abs/1904.08779)'s masking dropout rate, layer dropout, and the learning rate until training seemed to be stable enough. 

**Note**: When using this notebook to train XLSR-Wav2Vec2 on another language of Common Voice those hyper-parameter settings might not work very well. Feel free to adapt those depending on your use case. 

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_q.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_extractor()

In addition, let's enable gradient checkpointing to save some memory.

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./wav2vec2-large-xlsr-Viet",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  save_steps=40,
  eval_steps=40,
  logging_steps=40,
  learning_rate=3e-4,
  warmup_steps=50,
  save_total_limit=2,gradient_checkpointing=True,
  fp16=True #environnement non GPU = False(limité par utilisation du notebook )
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [ ]:
common_voice_train.shape

(221, 2)



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take multiple hours depending on the GPU allocated to this notebook. While the trained model yields somewhat satisfying results on *Common Voice*'s test data of Turkish, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how XLSR-Wav2Vec2's [checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) can be fine-tuned on a low-resource ASR dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.train() 

***** Running training *****
  Num examples = 221
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 210


Step,Training Loss,Validation Loss,Wer
40,18.978400,6.117734,1.000000
80,3.930800,3.598686,1.000000
120,3.449800,3.493193,1.000000
160,3.407100,3.433228,1.000000
200,3.396300,3.415799,1.000000


***** Running Evaluation *****
  Num examples = 198
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-Viet/checkpoint-40
Configuration saved in ./wav2vec2-large-xlsr-Viet/checkpoint-40/config.json
Model weights saved in ./wav2vec2-large-xlsr-Viet/checkpoint-40/pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-Viet/checkpoint-40/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 198
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-Viet/checkpoint-80
Configuration saved in ./wav2vec2-large-xlsr-Viet/checkpoint-80/config.json
Model weights saved in ./wav2vec2-large-xlsr-Viet/checkpoint-80/pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-Viet/checkpoint-80/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 198
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-Viet/checkpoint-120
Configuration saved in ./wav2vec2-large-xlsr-Viet/checkpoint-120/config.json
Model wei

TrainOutput(global_step=210, training_loss=6.477706327892485, metrics={'train_runtime': 434.255, 'train_samples_per_second': 15.268, 'train_steps_per_second': 0.484, 'total_flos': 8.839796995652198e+17, 'train_loss': 6.477706327892485, 'epoch': 30.0})

In [ ]:
model.save_pretrained("wav2vec2-large-xlsr-Viet")
processor.save_pretrained("wav2vec2-large-xlsr-Viet")

Configuration saved in wav2vec2-large-xlsr-Viet/config.json
Model weights saved in wav2vec2-large-xlsr-Viet/pytorch_model.bin
Feature extractor saved in wav2vec2-large-xlsr-Viet/preprocessor_config.json
tokenizer config file saved in wav2vec2-large-xlsr-Viet/tokenizer_config.json
Special tokens file saved in wav2vec2-large-xlsr-Viet/special_tokens_map.json
added tokens file saved in wav2vec2-large-xlsr-Viet/added_tokens.json
